In [288]:
import bisect

import numpy as np
import scipy.interpolate
import netCDF4
import matplotlib.pyplot as plt
import matplotlib.colors
import pycpt
import tqdm
import json
%matplotlib inline

In [20]:
urls = [
    'http://opendap.tudelft.nl/thredds/dodsC/data2/zandmotor/morphology/JETSKI/gridded/jetskikb118_3938.nc',
    'http://opendap.tudelft.nl/thredds/dodsC/data2/zandmotor/morphology/JETSKI/gridded/jetskikb118_3736.nc',
    'http://opendap.tudelft.nl/thredds/dodsC/data2/zandmotor/morphology/JETSKI/gridded/jetskikb117_3938.nc'
]


In [188]:
grids = []
for url in urls:
    ds = netCDF4.Dataset(url)
    times = ds.variables['time'][:]
    arrs = []
    z = ds.variables['z'][:]
    grids.append({
        "url": url,
        "z": z,
        "times": times
    })
    ds.close()

In [276]:
def fill(z):
    """fill z by space and then time"""

    def fill_space(z):
        """fill space"""
        assert len(z.shape) == 2, 'expected 2d, got %d d' % (len(z.shape), )
        # if we have no data...
        if z.mask.all():
            # nothing to interpolate, just return missings
            z_filled = np.ma.masked_all_like(z)
            return z_filled

        x = np.arange(z.shape[0])
        y = np.arange(z.shape[1])
        Y, X = np.meshgrid(y, x)
        points = np.c_[X.ravel(), Y.ravel()]
        F = scipy.interpolate.RegularGridInterpolator(
            (x, y),
            z.filled(np.nan),
            fill_value=np.nan
        )
        z_interp = F(np.c_[X.ravel(), Y.ravel()])
        z_filled = np.ma.masked_invalid(z_interp.reshape(z.shape))
        return z_filled

    def fill_time(z):
        """fill time"""
        assert len(z.shape) == 3, 'expected 3d, got %d d' % (len(z.shape), )

        z_filled = z.copy()
        z_filled.unshare_mask()
        for i in range(1, z_filled.shape[0]):
            arr_prev = z_filled[i-1]
            arr_current = z_filled[i]
            z_filled[i] = np.ma.where(arr_current.mask, arr_prev, arr_current)
        return z_filled

    filled_z = np.ma.row_stack([fill_space(z_i)[np.newaxis, ...] for z_i in z])
    filled_z = fill_time(filled_z)
    return filled_z

def interpolate(z, times):
    def calc(t):
        t_a_idx = bisect.bisect(times, t) - 1
        if t_a_idx < 0:
            t_a_idx = 0
        t_b_idx = t_a_idx + 1
        if t_b_idx > (len(times) - 1):
            t_b_idx = (len(times) - 1)
        if t_b_idx == t_a_idx:
            # images are equal, we're done
            img_a = z[t_a_idx]
            return img_a

        # time of frame before t 
        t_a = times[t_a_idx]
        # time of frame after t
        t_b = times[t_b_idx]
        # duration between two frames
        duration = t_b - t_a
        # compute fraction of time elapsed
        fraction = (t - t_a) / duration

        img_a = z[t_a_idx]
        img_b = z[t_b_idx]
        # if fraction is 0 we get image a
        # if fraction is 1 we get image b
        return img_a * (1 - fraction) + img_b * fraction
    return calc


In [115]:
# some colormaps I tried
cmap = pycpt.load.cmap_from_cptcity_url('wkp/shadowxfox/colombia.cpt')
cmap = pycpt.load.cmap_from_cptcity_url('grass/etopo2.cpt')
# somehow utah looks most like the moon....
cmap_topo = pycpt.load.cmap_from_cptcity_url('esri/hypsometry/na/utah_1.cpt')
# and this is goes nicely from dark to very light
cmap_bathy = pycpt.load.cmap_from_cptcity_url('xkcd/xkcd-bath.cpt')


In [125]:
# split into topo and bathy
sealevel = 0
N_bathy = matplotlib.colors.Normalize(vmin=-20, vmax=sealevel, clip=True)
N_topo = matplotlib.colors.Normalize(vmin=sealevel, vmax=20, clip=True)


# mix colors


In [278]:

for grid_i, grid in enumerate(grids): 
    filled_z = fill(grid['z'])
    Z = interpolate(filled_z, grid['times'])
    t_out = np.linspace(grid['times'][0], grid['times'][-1], num=400)
    for t_i, t in enumerate(t_out):
        arr = Z(t)
        # 2 colors
        bathy_colors = cmap_bathy(N_bathy(arr))
        topo_colors = cmap_topo(N_topo(arr))
        colors = np.where(np.repeat((arr < sealevel)[:,:,np.newaxis], 4, axis=2), bathy_colors, topo_colors)
        bw_mask = np.zeros_like(colors) + arr.mask[:, :, np.newaxis]
        img = np.ma.vstack([colors, bw_mask])
        plt.imsave('img_%02d_%06d.png' % (grid_i, t_i,), img)


/Users/baart_f/.virtualenvs/main/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in less
  # This is added back by InteractiveShellApp.init_path()


In [275]:
filled_z.shape, grid['times'].shape

((20, 625, 500), (57,))

In [291]:
metas = []
for i, url in enumerate(urls):
    ds = netCDF4.Dataset(url)
    times = netCDF4.num2date(ds.variables['time'][:].astype('double'), ds.variables['time'].units)
    x = ds.variables['x'][:]
    y = ds.variables['y'][:]
    Lat = ds.variables['lat'][:]
    Lon = ds.variables['lon'][:]
    metas.append({
        "url": url,
        "startTime": times[0].isoformat(),
        "endTime": times[-1].isoformat(),
        "extent": (float(Lon.min()), float(Lat.min()), float(Lon.max()), float(Lat.max())),
        "mp4": "jetski_%02d_yuv420.mp4" % (i, )
    })
    ds.close()
json.dump(metas, open('meta.json', 'w'))

array([15395, 15779, 15840, 15901, 15947, 16067, 16129, 16214, 16255,
       16316, 16373, 16476, 16578, 16639, 16648, 16706, 16872, 16928,
       17001, 17051], dtype=int32)